# Importing Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [2]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


# Thailand
TH = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/SETi.csv')
TH['Date'] = pd.to_datetime(TH['Date'])
TH = TH.rename(columns={'Price': 'Close'})
TH = TH.replace(',','', regex=True)
TH['Close'] = TH['Close'].astype(float, errors = 'raise')
mask = (TH['Date'] >= start_date) & (TH['Date'] <= end_date)
TH = TH.loc[mask]
TH = TH.set_index('Date')
TH = TH.sort_index(axis=0, ascending = False)

In [3]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

TH = get_returns(TH, 1)

,Open,High,Low,Close,Previous,Returns
Date,,,,,,
2020-12-30,1467.74,1479.04,1445.36,1449.35,1461.95,-0.865598
2020-12-29,1452.39,1468.60,1440.59,1461.95,1452.67,0.636792
2020-12-28,1505.15,1512.99,1452.48,1452.67,1486.31,-2.289330
2020-12-25,1463.32,1486.84,1462.77,1486.31,1451.52,2.368526
2020-12-24,1420.60,1453.87,1409.75,1451.52,1416.02,2.476116
...,...,...,...,...,...,...
2006-01-09,749.04,762.29,749.04,762.26,747.34,1.976747
2006-01-06,743.91,747.77,737.02,747.34,741.28,0.814181
2006-01-05,750.13,752.65,740.19,741.28,743.20,-0.258677


# Diebold-Mariano Test

In [4]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [5]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(country['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [6]:
TH_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/TH_VaR_FW-DN-0.01%25.csv')
TH_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/TH_VaR_FW-HS-0.01%25.csv')
TH_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/TH_VaR_FW-BRW-0.01%25.csv')
TH_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/TH_VaR_FW-G-0.01%25.csv')
TH_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/TH_VaR_FW-EA-0.01%25.csv')
TH_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/TH_VaR_FW-E-0.01%25.csv')
TH_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/TH_fw_FW-I-0.01%25.csv')
TH_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/TH_fw_FW-FI-0.01%25.csv')
TH_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/TH_fw_FW-FI0.633-0.01%25.csv')

In [7]:
# DN
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_garch, len(TH_fw_01_dn)))
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_earch, len(TH_fw_01_dn)))
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_egarch, len(TH_fw_01_dn)))
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_igarch, len(TH_fw_01_dn)))
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_figarch, len(TH_fw_01_dn)))
print(DM_test(TH, TH_fw_01_dn, TH_fw_01_figarch0633, len(TH_fw_01_dn)))

dm_return(DM=-1.1424343582667573, p_value=0.2536240723696228)
dm_return(DM=0.3794016805658664, p_value=0.704492868074927)
dm_return(DM=1.1678696603526408, p_value=0.24321619876985612)
dm_return(DM=-0.6502246816135312, p_value=0.515738562069451)
dm_return(DM=-0.49883340301736806, p_value=0.6180373991488857)
dm_return(DM=-0.821030091312683, p_value=0.4118799856396147)


In [8]:
# HS
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_garch, len(TH_fw_01_hs)))
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_earch, len(TH_fw_01_hs)))
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_egarch, len(TH_fw_01_hs)))
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_igarch, len(TH_fw_01_hs)))
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_figarch, len(TH_fw_01_hs)))
print(DM_test(TH, TH_fw_01_hs, TH_fw_01_figarch0633, len(TH_fw_01_hs)))

dm_return(DM=17.48739799762614, p_value=5.041278214706059e-58)
dm_return(DM=36.76701104199458, p_value=1.815873253671256e-172)
dm_return(DM=27.1563525195876, p_value=7.801678339630868e-115)
dm_return(DM=11.675194352262343, p_value=3.86106774807858e-29)
dm_return(DM=19.72866707687389, p_value=1.3515033346471189e-70)
dm_return(DM=16.63322871036649, p_value=2.121512131239208e-53)


In [9]:
# BRW
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_garch, len(TH_fw_01_brw)))
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_earch, len(TH_fw_01_brw)))
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_egarch, len(TH_fw_01_brw)))
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_igarch, len(TH_fw_01_brw)))
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_figarch, len(TH_fw_01_brw)))
print(DM_test(TH, TH_fw_01_brw, TH_fw_01_figarch0633, len(TH_fw_01_brw)))

dm_return(DM=7.097669199049889, p_value=2.8550055649473037e-12)
dm_return(DM=12.535157280225324, p_value=5.6981763789534085e-33)
dm_return(DM=11.655731491351172, p_value=4.691052599960987e-29)
dm_return(DM=4.881039880224469, p_value=1.2800690916365946e-06)
dm_return(DM=8.278175766517888, p_value=5.408691297511156e-16)
dm_return(DM=6.852216829525726, p_value=1.476654704894159e-11)


In [10]:
# GARCH
print(DM_test(TH, TH_fw_01_garch, TH_fw_01_earch, len(TH_fw_01_garch)))
print(DM_test(TH, TH_fw_01_garch, TH_fw_01_egarch, len(TH_fw_01_garch)))
print(DM_test(TH, TH_fw_01_garch, TH_fw_01_igarch, len(TH_fw_01_garch)))
print(DM_test(TH, TH_fw_01_garch, TH_fw_01_figarch, len(TH_fw_01_garch)))
print(DM_test(TH, TH_fw_01_garch, TH_fw_01_figarch0633, len(TH_fw_01_garch)))

dm_return(DM=1.509099181580403, p_value=0.13167838280142194)
dm_return(DM=6.056595878617475, p_value=2.160781037911541e-09)
dm_return(DM=0.14783553288047652, p_value=0.8825107727407324)
dm_return(DM=3.1258828190795853, p_value=0.0018384418999023771)
dm_return(DM=0.7835127176749375, p_value=0.43356392890884765)


In [11]:
# EARCH
print(DM_test(TH, TH_fw_01_earch, TH_fw_01_egarch, len(TH_fw_01_earch)))
print(DM_test(TH, TH_fw_01_earch, TH_fw_01_igarch, len(TH_fw_01_earch)))
print(DM_test(TH, TH_fw_01_earch, TH_fw_01_figarch, len(TH_fw_01_earch)))
print(DM_test(TH, TH_fw_01_earch, TH_fw_01_figarch0633, len(TH_fw_01_earch)))

dm_return(DM=1.0230743489527538, p_value=0.30658988265502274)
dm_return(DM=-0.9833306737971311, p_value=0.3257496270889819)
dm_return(DM=-0.786713581944615, p_value=0.43168850589421615)
dm_return(DM=-1.2095796705095818, p_value=0.22680669817222474)


In [12]:
# EGARCH
print(DM_test(TH, TH_fw_01_egarch, TH_fw_01_igarch, len(TH_fw_01_egarch)))
print(DM_test(TH, TH_fw_01_egarch, TH_fw_01_figarch, len(TH_fw_01_egarch)))
print(DM_test(TH, TH_fw_01_egarch, TH_fw_01_figarch0633, len(TH_fw_01_egarch)))

dm_return(DM=-1.6608615895504741, p_value=0.09714312467196236)
dm_return(DM=-4.5463934935628245, p_value=6.3240746582402265e-06)
dm_return(DM=-3.6430306029451196, p_value=0.0002872727982095731)


In [13]:
# IGARCH
print(DM_test(TH, TH_fw_01_igarch, TH_fw_01_figarch, len(TH_fw_01_igarch)))
print(DM_test(TH, TH_fw_01_igarch, TH_fw_01_figarch0633, len(TH_fw_01_igarch)))

dm_return(DM=0.550767129467312, p_value=0.5819510272429336)
dm_return(DM=0.16388212413716044, p_value=0.8698663942419549)


In [14]:
# FIGARCH
print(DM_test(TH, TH_fw_01_figarch, TH_fw_01_figarch0633, len(TH_fw_01_figarch)))

dm_return(DM=-1.6366625427650774, p_value=0.10210437876012211)


## 1.00 %

In [15]:
TH_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/TH_VaR_FW-DN-1%25.csv')
TH_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/TH_VaR_FW-HS-1%25.csv')
TH_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/TH_VaR_FW-BRW-1%25.csv')
TH_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/TH_VaR_FW-G-1%25.csv')
TH_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/TH_VaR_FW-EA-1%25.csv')
TH_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/TH_VaR_FW-E-1%25.csv')
TH_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/TH_fw_FW-I-1%25.csv')
TH_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/TH_fw_FW-FI-1%25.csv')
TH_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/TH_fw_FW-FI0.633-1%25.csv')

In [16]:
# DN
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_garch, len(TH_fw_1_dn)))
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_earch, len(TH_fw_1_dn)))
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_egarch, len(TH_fw_1_dn)))
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_igarch, len(TH_fw_1_dn)))
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_figarch, len(TH_fw_1_dn)))
print(DM_test(TH, TH_fw_1_dn, TH_fw_1_figarch0633, len(TH_fw_1_dn)))

dm_return(DM=-1.1422816162118217, p_value=0.2536874973136439)
dm_return(DM=0.3925533705198345, p_value=0.694756632416824)
dm_return(DM=1.1631231843377534, p_value=0.24513523038961196)
dm_return(DM=-0.6635624392161003, p_value=0.5071664735033725)
dm_return(DM=-0.5013502067063809, p_value=0.6162661385389513)
dm_return(DM=-0.8297261696967945, p_value=0.406947480866106)


In [17]:
# HS
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_garch, len(TH_fw_1_hs)))
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_earch, len(TH_fw_1_hs)))
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_egarch, len(TH_fw_1_hs)))
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_igarch, len(TH_fw_1_hs)))
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_figarch, len(TH_fw_1_hs)))
print(DM_test(TH, TH_fw_1_hs, TH_fw_1_figarch0633, len(TH_fw_1_hs)))

dm_return(DM=1.0004722317166164, p_value=0.317392295131435)
dm_return(DM=4.9925650596538285, p_value=7.354069315535796e-07)
dm_return(DM=4.236229320038497, p_value=2.5445094330770277e-05)
dm_return(DM=0.7411004287276594, p_value=0.45885560077362186)
dm_return(DM=1.8462064654182337, p_value=0.06524093994076209)
dm_return(DM=1.1854004022541702, p_value=0.23622018141037135)


In [18]:
# BRW
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_garch, len(TH_fw_1_brw)))
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_earch, len(TH_fw_1_brw)))
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_egarch, len(TH_fw_1_brw)))
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_igarch, len(TH_fw_1_brw)))
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_figarch, len(TH_fw_1_brw)))
print(DM_test(TH, TH_fw_1_brw, TH_fw_1_figarch0633, len(TH_fw_1_brw)))

dm_return(DM=1.466102439591177, p_value=0.1430233920350848)
dm_return(DM=4.066950011643035, p_value=5.247076791970544e-05)
dm_return(DM=4.61294714787544, p_value=4.639165138173229e-06)
dm_return(DM=1.0097990071640925, p_value=0.3129047384869618)
dm_return(DM=2.2922453882288627, p_value=0.022156972765526778)
dm_return(DM=1.5643825980284136, p_value=0.11813334197818141)


In [19]:
# GARCH
print(DM_test(TH, TH_fw_1_garch, TH_fw_1_earch, len(TH_fw_1_garch)))
print(DM_test(TH, TH_fw_1_garch, TH_fw_1_egarch, len(TH_fw_1_garch)))
print(DM_test(TH, TH_fw_1_garch, TH_fw_1_igarch, len(TH_fw_1_garch)))
print(DM_test(TH, TH_fw_1_garch, TH_fw_1_figarch, len(TH_fw_1_garch)))
print(DM_test(TH, TH_fw_1_garch, TH_fw_1_figarch0633, len(TH_fw_1_garch)))

dm_return(DM=1.5126461164324483, p_value=0.13077462124346784)
dm_return(DM=6.061313923820422, p_value=2.1011022577153787e-09)
dm_return(DM=0.12982030733310346, p_value=0.8967420659890122)
dm_return(DM=3.106861656829695, p_value=0.001959374202509549)
dm_return(DM=0.7569720776192185, p_value=0.4492951809563862)


In [20]:
# EARCH
print(DM_test(TH, TH_fw_1_earch, TH_fw_1_egarch, len(TH_fw_1_earch)))
print(DM_test(TH, TH_fw_1_earch, TH_fw_1_igarch, len(TH_fw_1_earch)))
print(DM_test(TH, TH_fw_1_earch, TH_fw_1_figarch, len(TH_fw_1_earch)))
print(DM_test(TH, TH_fw_1_earch, TH_fw_1_figarch0633, len(TH_fw_1_earch)))

dm_return(DM=1.0139719409051613, p_value=0.3109105619969982)
dm_return(DM=-1.0016780217510475, p_value=0.31680976418142254)
dm_return(DM=-0.7937436281738488, p_value=0.4275860950778718)
dm_return(DM=-1.2223059930921978, p_value=0.2219611601814415)


In [21]:
# EGARCH
print(DM_test(TH, TH_fw_1_egarch, TH_fw_1_igarch, len(TH_fw_1_egarch)))
print(DM_test(TH, TH_fw_1_egarch, TH_fw_1_figarch, len(TH_fw_1_egarch)))
print(DM_test(TH, TH_fw_1_egarch, TH_fw_1_figarch0633, len(TH_fw_1_egarch)))

dm_return(DM=-1.6814104080227064, p_value=0.09308353428051415)
dm_return(DM=-4.543317661492797, p_value=6.414673287717193e-06)
dm_return(DM=-3.6674692241419162, p_value=0.00026159160455476767)


In [22]:
# IGARCH
print(DM_test(TH, TH_fw_1_igarch, TH_fw_1_figarch, len(TH_fw_1_igarch)))
print(DM_test(TH, TH_fw_1_igarch, TH_fw_1_figarch0633, len(TH_fw_1_igarch)))

dm_return(DM=0.5670695062244091, p_value=0.5708301113164616)
dm_return(DM=0.17676199507354595, p_value=0.8597412284459303)


In [23]:
# FIGARCH
print(DM_test(TH, TH_fw_1_figarch, TH_fw_1_figarch0633, len(TH_fw_1_figarch)))

dm_return(DM=-1.6611862079760504, p_value=0.09707790853574119)


## 5.00 %

In [24]:
TH_fw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/TH_VaR_FW-DN-5%25.csv')
TH_fw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/TH_VaR_FW-HS-5%25.csv')
TH_fw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/TH_VaR_FW-BRW-5%25.csv')
TH_fw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/TH_VaR_FW-G-5%25.csv')
TH_fw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/TH_VaR_FW-EA-5%25.csv')
TH_fw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/TH_VaR_FW-E-5%25.csv')
TH_fw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-FW/TH_fw_FW-I-5%25.csv')
TH_fw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-FW/TH_fw_FW-FI-5%25.csv')
TH_fw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-FW/TH_fw_FW-FI0.633-5%25.csv')

In [25]:
# DN
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_garch, len(TH_fw_5_dn)))
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_earch, len(TH_fw_5_dn)))
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_egarch, len(TH_fw_5_dn)))
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_igarch, len(TH_fw_5_dn)))
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_figarch, len(TH_fw_5_dn)))
print(DM_test(TH, TH_fw_5_dn, TH_fw_5_figarch0633, len(TH_fw_5_dn)))

dm_return(DM=-1.1382607286889517, p_value=0.25536111956159213)
dm_return(DM=0.40646015380421957, p_value=0.6845160230805496)
dm_return(DM=1.150322084919121, p_value=0.25036378083576355)
dm_return(DM=-0.6764649384709176, p_value=0.4989460390303109)
dm_return(DM=-0.5022930226566467, p_value=0.6156031845096711)
dm_return(DM=-0.8364544347256618, p_value=0.4031554764596733)


In [26]:
# HS
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_garch, len(TH_fw_5_hs)))
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_earch, len(TH_fw_5_hs)))
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_egarch, len(TH_fw_5_hs)))
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_igarch, len(TH_fw_5_hs)))
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_figarch, len(TH_fw_5_hs)))
print(DM_test(TH, TH_fw_5_hs, TH_fw_5_figarch0633, len(TH_fw_5_hs)))

dm_return(DM=-4.585065753002717, p_value=5.28465875580966e-06)
dm_return(DM=-7.099518025809579, p_value=2.8193627832624085e-12)
dm_return(DM=-3.8068313565193477, p_value=0.00015179615092963359)
dm_return(DM=-2.9416940529049245, p_value=0.0033608851908553845)
dm_return(DM=-4.2769199299388365, p_value=2.1300962973513534e-05)
dm_return(DM=-4.083433323534087, p_value=4.89552161619572e-05)


In [27]:
# BRW
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_garch, len(TH_fw_5_brw)))
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_earch, len(TH_fw_5_brw)))
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_egarch, len(TH_fw_5_brw)))
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_igarch, len(TH_fw_5_brw)))
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_figarch, len(TH_fw_5_brw)))
print(DM_test(TH, TH_fw_5_brw, TH_fw_5_figarch0633, len(TH_fw_5_brw)))

dm_return(DM=-6.12327025642644, p_value=1.4519811408967332e-09)
dm_return(DM=-9.835184439669815, p_value=1.3461007069017313e-21)
dm_return(DM=-6.019906340235262, p_value=2.6848569717443834e-09)
dm_return(DM=-3.9110223251746987, p_value=9.991263192171343e-05)
dm_return(DM=-5.962841897318834, p_value=3.755107509204676e-09)
dm_return(DM=-5.493969010899228, p_value=5.323083210118257e-08)


In [28]:
# GARCH
print(DM_test(TH, TH_fw_5_garch, TH_fw_5_earch, len(TH_fw_5_garch)))
print(DM_test(TH, TH_fw_5_garch, TH_fw_5_egarch, len(TH_fw_5_garch)))
print(DM_test(TH, TH_fw_5_garch, TH_fw_5_igarch, len(TH_fw_5_garch)))
print(DM_test(TH, TH_fw_5_garch, TH_fw_5_figarch, len(TH_fw_5_garch)))
print(DM_test(TH, TH_fw_5_garch, TH_fw_5_figarch0633, len(TH_fw_5_garch)))

dm_return(DM=1.51225632550247, p_value=0.13087370385695823)
dm_return(DM=6.0620358066306, p_value=2.0921141969897936e-09)
dm_return(DM=0.10961475750118038, p_value=0.9127431007527003)
dm_return(DM=3.0806303093811898, p_value=0.0021381679799933333)
dm_return(DM=0.7264342103230395, p_value=0.467790558386769)


In [29]:
# EARCH
print(DM_test(TH, TH_fw_5_earch, TH_fw_5_egarch, len(TH_fw_5_earch)))
print(DM_test(TH, TH_fw_5_earch, TH_fw_5_igarch, len(TH_fw_5_earch)))
print(DM_test(TH, TH_fw_5_earch, TH_fw_5_figarch, len(TH_fw_5_earch)))
print(DM_test(TH, TH_fw_5_earch, TH_fw_5_figarch0633, len(TH_fw_5_earch)))

dm_return(DM=0.9986477471277969, p_value=0.318275061415829)
dm_return(DM=-1.0191680501066283, p_value=0.308439196096986)
dm_return(DM=-0.799066119398339, p_value=0.42449532193234407)
dm_return(DM=-1.2324921578358712, p_value=0.2181366584092756)


In [30]:
# EGARCH
print(DM_test(TH, TH_fw_5_egarch, TH_fw_5_igarch, len(TH_fw_5_egarch)))
print(DM_test(TH, TH_fw_5_egarch, TH_fw_5_figarch, len(TH_fw_5_egarch)))
print(DM_test(TH, TH_fw_5_egarch, TH_fw_5_figarch0633, len(TH_fw_5_egarch)))

dm_return(DM=-1.7020693790813972, p_value=0.08914083118102206)
dm_return(DM=-4.531178832838274, p_value=6.784516367678468e-06)
dm_return(DM=-3.690566926356187, p_value=0.00023931570045632357)


In [31]:
# IGARCH
print(DM_test(TH, TH_fw_5_igarch, TH_fw_5_figarch,  len(TH_fw_5_igarch)))
print(DM_test(TH, TH_fw_5_igarch, TH_fw_5_figarch0633, len(TH_fw_5_igarch)))

dm_return(DM=0.5841370771226854, p_value=0.559296984243357)
dm_return(DM=0.19076586764624456, p_value=0.8487586717177527)


In [32]:
# FIGARCH
print(DM_test(TH, TH_fw_5_figarch, TH_fw_5_figarch0633, len(TH_fw_5_figarch)))

dm_return(DM=-1.6844666120810496, p_value=0.09249158002221897)


# Rolling Window

## 0.01 %

In [33]:
TH_rw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/TH_VaR_RW-DN-0.01%25.csv')
TH_rw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/TH_VaR_RW-HS-0.01%25.csv')
TH_rw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/TH_VaR_RW-BRW-0.01%25.csv')
TH_rw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/TH_VaR_RW-G-0.01%25.csv')
TH_rw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/TH_VaR_RW-EA-0.01%25.csv')
TH_rw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/TH_VaR_RW-E-0.01%25.csv')
TH_rw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-RW/TH_RW-I-0.01%25.csv')
TH_rw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-RW/TH_RW-FI-0.01%25.csv')
TH_rw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-RW/TH_RW-FI0.633-0.01%25.csv')

In [34]:
# DN
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_garch, len(TH_rw_01_dn)))
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_earch, len(TH_rw_01_dn)))
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_egarch, len(TH_rw_01_dn)))
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_igarch, len(TH_rw_01_dn)))
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_figarch, len(TH_rw_01_dn)))
print(DM_test(TH, TH_rw_01_dn, TH_rw_01_figarch0633, len(TH_rw_01_dn)))

dm_return(DM=-0.9195140839976237, p_value=0.35811101104330334)
dm_return(DM=0.8924240470299659, p_value=0.3724409169699666)
dm_return(DM=2.14425279484971, p_value=0.032321070605146975)
dm_return(DM=-0.5097756657763967, p_value=0.6103528384016379)
dm_return(DM=-0.18697591625730547, p_value=0.851728137853471)
dm_return(DM=-0.6537044200611505, p_value=0.5134949175122976)


In [35]:
# HS
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_garch, len(TH_rw_01_hs)))
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_earch, len(TH_rw_01_hs)))
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_egarch, len(TH_rw_01_hs)))
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_igarch, len(TH_rw_01_hs)))
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_figarch, len(TH_rw_01_hs)))
print(DM_test(TH, TH_rw_01_hs, TH_rw_01_figarch0633, len(TH_rw_01_hs)))

dm_return(DM=48.76972849311587, p_value=4.3375105217563275e-239)
dm_return(DM=97.67458783067663, p_value=0.0)
dm_return(DM=76.55556469769732, p_value=0.0)
dm_return(DM=28.447352234016904, p_value=1.1311144726883377e-122)
dm_return(DM=53.81028294430307, p_value=8.886753368791012e-265)
dm_return(DM=40.14935014892526, p_value=5.537619380207112e-192)


In [36]:
# BRW
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_garch, len(TH_rw_01_brw)))
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_earch, len(TH_rw_01_brw)))
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_egarch, len(TH_rw_01_brw)))
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_igarch, len(TH_rw_01_brw)))
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_figarch, len(TH_rw_01_brw)))
print(DM_test(TH, TH_rw_01_brw, TH_rw_01_figarch0633, len(TH_rw_01_brw)))

dm_return(DM=6.5307461294107405, p_value=1.1793638205087156e-10)
dm_return(DM=12.314109184835075, p_value=5.72508443595821e-32)
dm_return(DM=12.271757140695696, p_value=8.88028671822116e-32)
dm_return(DM=3.8736342327253603, p_value=0.00011622121552209964)
dm_return(DM=7.821187796229173, p_value=1.6987684666577288e-14)
dm_return(DM=5.465813654325098, p_value=6.204490146815766e-08)


In [37]:
# GARCH
print(DM_test(TH, TH_rw_01_garch, TH_rw_01_earch, len(TH_rw_01_garch)))
print(DM_test(TH, TH_rw_01_garch, TH_rw_01_egarch, len(TH_rw_01_garch)))
print(DM_test(TH, TH_rw_01_garch, TH_rw_01_igarch, len(TH_rw_01_garch)))
print(DM_test(TH, TH_rw_01_garch, TH_rw_01_figarch, len(TH_rw_01_garch)))
print(DM_test(TH, TH_rw_01_garch, TH_rw_01_figarch0633, len(TH_rw_01_garch)))

dm_return(DM=1.5402048612851391, p_value=0.1239159898473886)
dm_return(DM=5.454918194664948, p_value=6.582288558634434e-08)
dm_return(DM=0.007840590610589998, p_value=0.9937461827317571)
dm_return(DM=2.9275720581020988, p_value=0.003515588838782084)
dm_return(DM=0.1837748887694543, p_value=0.8542378194745938)


In [38]:
# EARCH
print(DM_test(TH, TH_rw_01_earch, TH_rw_01_egarch, len(TH_rw_01_earch)))
print(DM_test(TH, TH_rw_01_earch, TH_rw_01_igarch, len(TH_rw_01_earch)))
print(DM_test(TH, TH_rw_01_earch, TH_rw_01_figarch, len(TH_rw_01_earch)))
print(DM_test(TH, TH_rw_01_earch, TH_rw_01_figarch0633, len(TH_rw_01_earch)))

dm_return(DM=1.721858508544426, p_value=0.0854916124899605)
dm_return(DM=-0.9152220578374678, p_value=0.3603579133300956)
dm_return(DM=-0.7983015398307451, p_value=0.4249385072037922)
dm_return(DM=-1.2752871458586388, p_value=0.20258732550735165)


In [39]:
# EGARCH
print(DM_test(TH, TH_rw_01_egarch, TH_rw_01_igarch, len(TH_rw_01_egarch)))
print(DM_test(TH, TH_rw_01_egarch, TH_rw_01_figarch, len(TH_rw_01_egarch)))
print(DM_test(TH, TH_rw_01_egarch, TH_rw_01_figarch0633, len(TH_rw_01_egarch)))

dm_return(DM=-1.5543227130142714, p_value=0.12051308950023187)
dm_return(DM=-4.114608079467531, p_value=4.2909015149903384e-05)
dm_return(DM=-2.7710486395839053, p_value=0.005720616640940641)


In [40]:
# IGARCH
print(DM_test(TH, TH_rw_01_igarch, TH_rw_01_figarch,  len(TH_rw_01_igarch)))
print(DM_test(TH, TH_rw_01_igarch, TH_rw_01_figarch0633, len(TH_rw_01_igarch)))

dm_return(DM=0.6526360003274027, p_value=0.5141832646744993)
dm_return(DM=0.1352764692912163, p_value=0.8924281899271245)


In [41]:
# FIGARCH
print(DM_test(TH, TH_rw_01_figarch, TH_rw_01_figarch0633, len(TH_rw_01_figarch)))

dm_return(DM=-1.4916334362649586, p_value=0.13619957712364963)


## 1.00 %

In [42]:
TH_rw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/TH_VaR_RW-DN-1%25.csv')
TH_rw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/TH_VaR_RW-HS-1%25.csv')
TH_rw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/TH_VaR_RW-BRW-1%25.csv')
TH_rw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/TH_VaR_RW-G-1%25.csv')
TH_rw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/TH_VaR_RW-EA-1%25.csv')
TH_rw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/TH_VaR_RW-E-1%25.csv')
TH_rw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-RW/TH_RW-I-1%25.csv')
TH_rw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-RW/TH_RW-FI-1%25.csv')
TH_rw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-RW/TH_RW-FI0.633-1%25.csv')

In [43]:
# DN
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_garch, len(TH_rw_1_dn)))
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_earch, len(TH_rw_1_dn)))
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_egarch, len(TH_rw_1_dn)))
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_igarch, len(TH_rw_1_dn)))
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_figarch, len(TH_rw_1_dn)))
print(DM_test(TH, TH_rw_1_dn, TH_rw_1_figarch0633, len(TH_rw_1_dn)))

dm_return(DM=-0.9188672268566606, p_value=0.3584490784128064)
dm_return(DM=0.909855609714252, p_value=0.3631797252884882)
dm_return(DM=2.125233215945512, p_value=0.033880880749362526)
dm_return(DM=-0.5169716594219813, p_value=0.6053224959537973)
dm_return(DM=-0.19223510326402818, p_value=0.847608086116578)
dm_return(DM=-0.6657568758023351, p_value=0.5057633419717249)


In [44]:
# HS
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_garch, len(TH_rw_1_hs)))
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_earch, len(TH_rw_1_hs)))
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_egarch, len(TH_rw_1_hs)))
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_igarch, len(TH_rw_1_hs)))
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_figarch, len(TH_rw_1_hs)))
print(DM_test(TH, TH_rw_1_hs, TH_rw_1_figarch0633, len(TH_rw_1_hs)))

dm_return(DM=0.9124732121921237, p_value=0.3618015981087509)
dm_return(DM=5.075400528036105, p_value=4.838193335106687e-07)
dm_return(DM=5.068675881329981, p_value=5.006596692124439e-07)
dm_return(DM=0.5137446626499044, p_value=0.607576019680867)
dm_return(DM=1.8183372587701403, p_value=0.06939601760819207)
dm_return(DM=0.8140283709901702, p_value=0.4158771338525613)


In [45]:
# BRW
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_garch, len(TH_rw_1_brw)))
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_earch, len(TH_rw_1_brw)))
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_egarch, len(TH_rw_1_brw)))
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_igarch, len(TH_rw_1_brw)))
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_figarch, len(TH_rw_1_brw)))
print(DM_test(TH, TH_rw_1_brw, TH_rw_1_figarch0633, len(TH_rw_1_brw)))

dm_return(DM=0.984693078121021, p_value=0.32508019167223023)
dm_return(DM=3.53047685055298, p_value=0.00043914480965423844)
dm_return(DM=4.720304810664276, p_value=2.7912961569683703e-06)
dm_return(DM=0.5545715882584532, p_value=0.5793467012995394)
dm_return(DM=1.8678978691206036, p_value=0.06215108074774161)
dm_return(DM=0.8646936707898364, p_value=0.38747278937025886)


In [46]:
# GARCH
print(DM_test(TH, TH_rw_1_garch, TH_rw_1_earch, len(TH_rw_1_garch)))
print(DM_test(TH, TH_rw_1_garch, TH_rw_1_egarch, len(TH_rw_1_garch)))
print(DM_test(TH, TH_rw_1_garch, TH_rw_1_igarch, len(TH_rw_1_garch)))
print(DM_test(TH, TH_rw_1_garch, TH_rw_1_figarch, len(TH_rw_1_garch)))
print(DM_test(TH, TH_rw_1_garch, TH_rw_1_figarch0633, len(TH_rw_1_garch)))

dm_return(DM=1.5417334174396977, p_value=0.12354396600388465)
dm_return(DM=5.452019599240168, p_value=6.686499433070258e-08)
dm_return(DM=-0.0010220813425232873, p_value=0.9991847585562256)
dm_return(DM=2.899451451453552, p_value=0.0038431352150019374)
dm_return(DM=0.16026721444086553, p_value=0.8727120791014062)


In [47]:
# EARCH
print(DM_test(TH, TH_rw_1_earch, TH_rw_1_egarch, len(TH_rw_1_earch)))
print(DM_test(TH, TH_rw_1_earch, TH_rw_1_igarch, len(TH_rw_1_earch)))
print(DM_test(TH, TH_rw_1_earch, TH_rw_1_figarch, len(TH_rw_1_earch)))
print(DM_test(TH, TH_rw_1_earch, TH_rw_1_figarch0633, len(TH_rw_1_earch)))

dm_return(DM=1.700625726549911, p_value=0.08941189518559524)
dm_return(DM=-0.9257285288621326, p_value=0.3548733992080575)
dm_return(DM=-0.8089897371076872, p_value=0.41876774008455164)
dm_return(DM=-1.29285781676807, p_value=0.19644304998041995)


In [48]:
# EGARCH
print(DM_test(TH, TH_rw_1_egarch, TH_rw_1_igarch, len(TH_rw_1_egarch)))
print(DM_test(TH, TH_rw_1_egarch, TH_rw_1_figarch, len(TH_rw_1_egarch)))
print(DM_test(TH, TH_rw_1_egarch, TH_rw_1_figarch0633, len(TH_rw_1_egarch)))

dm_return(DM=-1.5623719921555232, p_value=0.11860599042744979)
dm_return(DM=-4.134382735372276, p_value=3.9449242588764514e-05)
dm_return(DM=-2.7926121824168595, p_value=0.005356447765135085)


In [49]:
# IGARCH
print(DM_test(TH, TH_rw_1_igarch, TH_rw_1_figarch,  len(TH_rw_1_igarch)))
print(DM_test(TH, TH_rw_1_igarch, TH_rw_1_figarch0633, len(TH_rw_1_igarch)))

dm_return(DM=0.6578734824159712, p_value=0.5108135338364406)
dm_return(DM=0.13192801898930076, p_value=0.8950752488890814)


In [50]:
# FIGARCH
print(DM_test(TH, TH_rw_1_figarch, TH_rw_1_figarch0633, len(TH_rw_1_figarch)))

dm_return(DM=-1.5106351183853162, p_value=0.1312864313880917)


## 5.00 %

In [51]:
TH_rw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/TH_VaR_RW-DN-5%25.csv')
TH_rw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/TH_VaR_RW-HS-5%25.csv')
TH_rw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/TH_VaR_RW-BRW-5%25.csv')
TH_rw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/TH_VaR_RW-G-5%25.csv')
TH_rw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/TH_VaR_RW-EA-5%25.csv')
TH_rw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/TH_VaR_RW-E-5%25.csv')
TH_rw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-RW/TH_RW-I-5%25.csv')
TH_rw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-RW/TH_RW-FI-5%25.csv')
TH_rw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-RW/TH_RW-FI0.633-5%25.csv')

In [52]:
# DN
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_garch, len(TH_rw_5_dn)))
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_earch, len(TH_rw_5_dn)))
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_egarch, len(TH_rw_5_dn)))
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_igarch, len(TH_rw_5_dn)))
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_figarch, len(TH_rw_5_dn)))
print(DM_test(TH, TH_rw_5_dn, TH_rw_5_figarch0633, len(TH_rw_5_dn)))

dm_return(DM=-0.9153031821237306, p_value=0.3603153622288304)
dm_return(DM=0.9270392766133241, p_value=0.3541928952250164)
dm_return(DM=2.088300822683342, p_value=0.03709447736799834)
dm_return(DM=-0.5238874950550466, p_value=0.6005056134853131)
dm_return(DM=-0.19744193646061994, p_value=0.8435331544022694)
dm_return(DM=-0.6773404204826762, p_value=0.49839083385117333)


In [53]:
# HS
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_garch, len(TH_rw_5_hs)))
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_earch, len(TH_rw_5_hs)))
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_egarch, len(TH_rw_5_hs)))
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_igarch, len(TH_rw_5_hs)))
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_figarch, len(TH_rw_5_hs)))
print(DM_test(TH, TH_rw_5_hs, TH_rw_5_figarch0633, len(TH_rw_5_hs)))

dm_return(DM=-4.629723456912506, p_value=4.287987679367e-06)
dm_return(DM=-7.642761214931756, p_value=6.246318898426878e-14)
dm_return(DM=-3.951625454231922, p_value=8.466314277690475e-05)
dm_return(DM=-2.613075244658367, p_value=0.009145987197104318)
dm_return(DM=-4.2901098662162465, p_value=2.0101845581685027e-05)
dm_return(DM=-3.6833164971401455, p_value=0.0002461085396595393)


In [54]:
# BRW
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_garch, len(TH_rw_5_brw)))
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_earch, len(TH_rw_5_brw)))
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_egarch, len(TH_rw_5_brw)))
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_igarch, len(TH_rw_5_brw)))
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_figarch, len(TH_rw_5_brw)))
print(DM_test(TH, TH_rw_5_brw, TH_rw_5_figarch0633, len(TH_rw_5_brw)))

dm_return(DM=-6.413054769169329, p_value=2.4702565344596266e-10)
dm_return(DM=-11.092129377265922, p_value=1.1944930624822363e-26)
dm_return(DM=-6.901804858514769, p_value=1.0636799282294127e-11)
dm_return(DM=-3.6011142937613982, p_value=0.0003369075318074936)
dm_return(DM=-6.2920276585768296, p_value=5.220538354886984e-10)
dm_return(DM=-5.1124361552966775, p_value=4.004412126136718e-07)


In [55]:
# GARCH
print(DM_test(TH, TH_rw_5_garch, TH_rw_5_earch, len(TH_rw_5_garch)))
print(DM_test(TH, TH_rw_5_garch, TH_rw_5_egarch, len(TH_rw_5_garch)))
print(DM_test(TH, TH_rw_5_garch, TH_rw_5_igarch, len(TH_rw_5_garch)))
print(DM_test(TH, TH_rw_5_garch, TH_rw_5_figarch, len(TH_rw_5_garch)))
print(DM_test(TH, TH_rw_5_garch, TH_rw_5_figarch0633, len(TH_rw_5_garch)))

dm_return(DM=1.5394247429628898, p_value=0.1241061943047165)
dm_return(DM=5.445251815396209, p_value=6.936088844200707e-08)
dm_return(DM=-0.010866738578356355, p_value=0.9913325463944787)
dm_return(DM=2.8634913507881063, p_value=0.004302409295120166)
dm_return(DM=0.13393551246843508, p_value=0.8934881174926079)


In [56]:
# EARCH
print(DM_test(TH, TH_rw_5_earch, TH_rw_5_egarch, len(TH_rw_5_earch)))
print(DM_test(TH, TH_rw_5_earch, TH_rw_5_igarch, len(TH_rw_5_earch)))
print(DM_test(TH, TH_rw_5_earch, TH_rw_5_figarch, len(TH_rw_5_earch)))
print(DM_test(TH, TH_rw_5_earch, TH_rw_5_figarch0633, len(TH_rw_5_earch)))

dm_return(DM=1.6670233887202248, p_value=0.09591118865643442)
dm_return(DM=-0.9356819489298369, p_value=0.34972655864785396)
dm_return(DM=-0.8183886408791808, p_value=0.41338525006088744)
dm_return(DM=-1.3091039795605959, p_value=0.1908846519948985)


In [57]:
# EGARCH
print(DM_test(TH, TH_rw_5_egarch, TH_rw_5_igarch, len(TH_rw_5_egarch)))
print(DM_test(TH, TH_rw_5_egarch, TH_rw_5_figarch, len(TH_rw_5_egarch)))
print(DM_test(TH, TH_rw_5_egarch, TH_rw_5_figarch0633, len(TH_rw_5_egarch)))

dm_return(DM=-1.5696993749388362, p_value=0.11689061838166782)
dm_return(DM=-4.150943752182931, p_value=3.6757476853246906e-05)
dm_return(DM=-2.8142174384422836, p_value=0.005012738495597444)


In [58]:
# IGARCH
print(DM_test(TH, TH_rw_5_igarch, TH_rw_5_figarch,  len(TH_rw_5_igarch)))
print(DM_test(TH, TH_rw_5_igarch, TH_rw_5_figarch0633, len(TH_rw_5_igarch)))

dm_return(DM=0.6627153283271883, p_value=0.5077086671822887)
dm_return(DM=0.12803232711550594, p_value=0.8981563919318181)


In [59]:
# FIGARCH
print(DM_test(TH, TH_rw_5_figarch, TH_rw_5_figarch0633, len(TH_rw_5_figarch)))

dm_return(DM=-1.529030700586615, p_value=0.12666225003361145)
